<a href="https://colab.research.google.com/github/Aman2568/codealpha_tasks/blob/main/ObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Object Detection and Tracking

Develop a system capable of detecting and
tracking objects in real-time video streams. Use
deep learning models like YOLO (You Only Look
Once) or Faster R-CNN for accurate object
detection and tracking.We will use YOLO due to its balance of speed and accuracy, making it suitable for real-time applications.**

***Installing the necessary Libraries***

In [ ]:
!pip install opencv-python opencv-python-headless numpy torch torchvision


***We will use a pre-trained YOLOv3 model.  Downloading the weights and configuration files for a smaller and faster version like YOLOv3-tiny.***

In [ ]:
!wget https://pjreddie.com/media/files/yolov3-tiny.cfg
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2024-07-10 15:27:24--  https://pjreddie.com/media/files/yolov3-tiny.cfg
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-07-10 15:27:24 ERROR 404: Not Found.

--2024-07-10 15:27:24--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  23.5MB/s    in 1.4s    

2024-07-10 15:27:26 (23.5 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15851, done.
remote: Total 15851 (delta 0), reused 0 (delta 0), pack-reused 15851
Receiving objects: 100% (15851/15851), 14.37 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (10683/10683), done.


In [ ]:
%cd darknet
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3.cfg

/content/darknet
--2024-07-10 15:32:41--  https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2024-07-10 15:32:41 (76.9 MB/s) - ‘yolov3.cfg’ saved [8342/8342]



***Loading the YOLO Model in Python***

In [ ]:
pwd

'/content/darknet'

In [ ]:
%cd /content/

/content


In [ ]:
import cv2
import numpy as np


net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')#loading YOLO


with open('coco.names', 'r') as f:               #loading COCO class label
    classes = f.read().strip().split('\n')

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


***Defining a function to perform object detection on a frame from the video stream.***

In [ ]:
def detect_objects(frame):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    result = []
    for i in range(len(boxes)):
        if i in indexes:
            box = boxes[i]
            result.append((class_ids[i], confidences[i], box))

    return result

***Displaying detected ojects***

In [ ]:
def draw_boxes(frame, detections):
    for (class_id, confidence, box) in detections:
        x, y, w, h = box
        label = str(classes[class_id])
        color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame

***Finally, we willll capture video from a webcam -> perform object detection -> and display the results in real-time.***

In [ ]:
def main():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_objects(frame)
        frame = draw_boxes(frame, detections)

        cv2.imshow('Object Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()